In [ ]:
import os
spark_home = os.path.abspath(os.getcwd() + "/../spark-3.5.5-bin-hadoop3")
hadoop_home = os.path.abspath(os.getcwd() + "/../winutils")
print(f"I am using the following SPARK_HOME: {spark_home}")
if os.name == 'nt':
    os.environ["HADOOP_HOME"] = f"{hadoop_home}"
    print(f"Windows detected: set HADOOP_HOME to: {os.environ['HADOOP_HOME']}")
    hadoop_bin = os.path.join(hadoop_home, "bin")
    os.environ["PATH"] = f"{hadoop_bin};{os.environ['PATH']}"
    print(f"  Also added Hadoop bin directory to PATH: {hadoop_bin}")

import findspark
import pyspark
from pyspark.streaming import StreamingContext

findspark.init(spark_home)
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [ ]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [ ]:
# Save less frequently and create less subdirectories. I.e. every 120 seconds
ssc = StreamingContext(sc, 120)

In [ ]:
lines = ssc.socketTextStream("seppe.net", 7778)

In [ ]:
out_dir = f"{os.path.abspath('')}{os.path.sep}saved_data"
lines.saveAsTextFiles(f"file:///{out_dir}")
print("Saving to", out_dir)

In [ ]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

In [ ]:
# Don't run this cell unless you want to stop. You should see subdirectories appear in the out_dir
ssc_t.stop()